<a href="https://colab.research.google.com/github/Hubert26/machine-learning/blob/data-exp/project_hubert_szewczyk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OPIS PROJEKTU

Lending Club to firma pożyczkowa typu peer-to-peer, która łączy pożyczkobiorców z
inwestorami za pośrednictwem platformy internetowej. Obsługuje osoby, które potrzebują
pożyczek osobistych w wysokości od 1000 do 40 000 USD. Pożyczkobiorcy otrzymują pełną
kwotę udzielonej pożyczki pomniejszoną o opłatę początkową, która jest uiszczana firmie.
Inwestorzy kupują weksle zabezpieczone osobistymi pożyczkami i płacą Lending Club
opłatę za usługę. Firma Lending Club udostępnia dane o wszystkich pożyczkach
udzielonych za pośrednictwem swojej platformy w określonych okresach.
Na potrzeby tego projektu zostały użyte dane dotyczące pożyczek udzielonych za
pośrednictwem Lending Club na przestrzeni lat 2007 -2011. Każda pożyczka jest opatrzona
informacją o tym, czy ostatecznie została spłacona (Fully Paid lub Charged off w kolumnie
loan_status).

W tym projekcie chcę zbudować model klasyfikacyjny, który na podstawie zebranych danych będzie przewidywał, czy potencjalny pozyczkobiorca spłaci swój dług.

#KOD

##Import bibliotek i ustawienia

In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

#Ustawienie braku maksymalnej ilości wyświetlanych kolumn
pd.options.display.max_columns = None

##Definicje funkcji

In [3]:
def display_info(df, selected_columns, df_info):
    for col_name in selected_columns:
        print("Column Name:", col_name)

        # Description
        if col_name in df_info.index:
            print("Description:", df_info.loc[col_name]['Description'])
        else:
            print("Description not available.")

        # Statistics
        print("\nStatistics:")
        print(df[col_name].describe())

        # Mode
        print("\nMode:")
        print(df[col_name].mode())

        # NaN count
        nan_count = df[col_name].isna().sum()
        print("\nNaN count:", nan_count)

        # Unique values len
        print("\nUnique values len:")
        print(len(df[col_name].unique()))

        # Unique values
        print("\nUnique values:")
        print(df[col_name].unique())

        # Value counts
        print("\nValue counts:")
        print(df[col_name].value_counts())

        print("\n" + "=" * 40)  # Separator between columns

In [48]:
def nan_counts(df, min_nan, max_nan):
    mask = df.isna()
    nan_counts_columns = mask.sum()
    naan_column = pd.DataFrame(nan_counts_columns, columns=['NaN'])
    selected_columns = [col_name for col_name in naan_column.index if min_nan <= naan_column.at[col_name, 'NaN'] <= max_nan]

    #display(naan_column.loc[:])
    return selected_columns

In [50]:
def unique_counts(df, min_unique=0, max_unique=None):

    unique_counts_columns = df.nunique()
    unique_column = pd.DataFrame(unique_counts_columns, columns=['Unique'])
    selected_columns = [col_name for col_name, unique_count in unique_column.iterrows()
                        if min_unique <= unique_count['Unique'] and (max_unique is None or unique_count['Unique'] <= max_unique)]

    return selected_columns

In [57]:
df[unique_counts(df, 0, 1)]

,member_id,mths_since_last_major_derog,annual_inc_joint,dti_joint,verification_status_joint,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [53]:
unique_counts_columns = df.nunique()
unique_counts_columns

id                                            42536
member_id                                         0
loan_amnt                                       898
int_rate                                        394
installment                                   16459
                                              ...  
hardship_dpd                                      0
hardship_loan_status                              0
orig_projected_additional_accrued_interest        0
hardship_payoff_balance_amount                    0
hardship_last_payment_amount                      0
Length: 117, dtype: int64

In [6]:
def diff_counts(df, min_diff, max_diff):
  unique_value_counts = {}
  for col_name in df.columns.tolist():
      unique_value_counts[col_name] = df[col_name].value_counts()

  #Obliczenie różnicy między liczbami wystąpień unikalnych wartości w kolumnach
  value_count_differences = pd.DataFrame(index=df.columns.tolist(), columns=['diff'])
  for col_name, counts in unique_value_counts.items():
  # Sortowanie wartości występujących w danej kolumnie malejąco
      sorted_counts = counts.sort_values(ascending=False)

      # Sprawdzanie, czy są przynajmniej dwie różne wartości
      if len(sorted_counts) >= 2:
          diff = sorted_counts.iloc[0] - sorted_counts.iloc[1]
          value_count_differences.at[col_name, 'diff'] = diff
      else:
          value_count_differences.at[col_name, 'diff'] = 0  # Jeśli jest mniej niż dwie różne wartości, to różnica wynosi 0

  selected_columns = value_count_differences.index[
      (value_count_differences['diff'] >= min_diff) &
       (value_count_differences['diff'] <= max_diff)].tolist()
  return selected_columns


##1. Obróbka danych

###Pobranie danych z plików

In [7]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/Hubert26/machine-learning/main/Loan_data.csv', low_memory=False)

In [8]:
df = df_raw.copy()

In [9]:
df_info_raw = pd.read_csv('https://raw.githubusercontent.com/Hubert26/machine-learning/main/LCDataDictionary.csv')

In [10]:
df_info = df_info_raw.iloc[:, 0:2]
df_info.set_index(df_info.columns.tolist()[0], inplace=True)

###Eksploracja danych

In [11]:
df.shape

(42536, 151)

In [12]:
df.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1077501,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,NaN,NaN,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.0,0.0,0.00,Jan-2015,171.62,NaN,Aug-2018,739.0,735.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,740.0,744.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.4%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.0,122.9,1.11,Apr-2013,119.66,NaN,Oct-2016,499.0,0.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42536 entries, 0 to 42535
Columns: 151 entries, id to settlement_term
dtypes: float64(120), object(31)
memory usage: 49.0+ MB


In [14]:
df.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,42535.000000,42535.000000,42535.000000,42535.000000,4.253100e+04,42535.000000,42506.000000,42535.000000,42535.000000,42506.000000,15609.000000,3651.000000,42506.000000,42506.000000,4.253500e+04,42506.000000,42535.0,42535.0,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42390.0,0.0,42535.0,0.0,0.0,0.0,42506.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42390.0,42506.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41170.000000,42430.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.000000,160.000000,160.000000
mean,NaN,11089.722581,10821.585753,10139.938785,322.623063,6.913656e+04,13.373043,0.152449,713.052545,717.052545,1.081424,35.017618,59.175568,9.343951,0.058156,1.429786e+04,22.124406,0.0,0.0,12020.189644,11312.729377,9675.675936,2239.981444,1.516890,103.015467,14.387843,2613.245652,689.922511,676.952039,0.0,NaN,1.0,NaN,NaN,NaN,0.000094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.143039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045227,0.000024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4272.137875,49.905875,1.200000
std,NaN,7410.938391,7146.914675,7131.598014,208.927216,6.409635e+04,6.726315,0.512406,36.188439,36.188439,1.527455,22.418427,47.145396,4.496274,0.245713,2.201844e+04,11.592811,0.0,0.0,9094.685888,9038.506549,7105.750304,2585.057393,7.830064,735.799086,162.177837,4385.066535,80.818099,119.647752,0.0,NaN,0.0,NaN,NaN,NaN,0.009700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,29.359579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [15]:
#Sprawdzenie czy wszystkie wartości są unikalne
len(df['id'].unique())==df.shape[0]

True

In [16]:
column_names = df.columns.tolist()

In [17]:
df_info

,Description
LoanStatNew,
acc_now_delinq,The number of accounts on which the borrower i...
acc_open_past_24mths,Number of trades opened in past 24 months.
addr_state,The state provided by the borrower in the loan...
all_util,Balance to credit limit on all trades
annual_inc,The self-reported annual income provided by th...
...,...
settlement_amount,The loan amount that the borrower has agreed t...
settlement_percentage,The settlement amount as a percentage of the p...
settlement_term,The number of months that the borrower will be...


###Wyświetlenie Loan Status

Charged Off to status długu uznawanego za nieściągalny po 180 dniach braku płatności, ale nie oznacza umorzenia długu. Odpisanie długu wpływa negatywnie na historię kredytową i zdolność kredytową pożyczkobiorcy. Mimo spłaty długu, status Charged Off pozostaje w raporcie kredytowym przez siedem lat, choć może zostać zmieniony na "odpis opłacony" lub "odpis rozliczony". W praktyce, wiele osób spłaciło dług, ale nadal ma status Charged Off.

In [18]:
df_info.loc["settlement_status"]['Description']

'The status of the borrower’s settlement plan. Possible values are: COMPLETE, ACTIVE, BROKEN, CANCELLED, DENIED, DRAFT'

In [19]:
df.settlement_status.unique()

array([nan, 'COMPLETE', 'BROKEN', 'ACTIVE'], dtype=object)

In [20]:
df_info.loc["loan_status"]['Description']

'Current status of the loan'

In [21]:
df.loan_status.unique()

array(['Fully Paid', 'Charged Off', nan,
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [22]:
df["loan_status"] = df["loan_status"].replace(
    {"Does not meet the credit policy. Status:Fully Paid": "Fully Paid",
     "Does not meet the credit policy. Status:Charged Off": "Charged Off"})

In [23]:
df[(df.loan_status == 'Charged Off') & (df.settlement_status == "COMPLETE")].shape[0]

142

Wynika z tego, że są 142 wyniki z loan_status == Charged Off wraz z settlement_status == COMPLETE.

W dalszej części zmieniamy wynik Charged Off na Fully Paid dla takich wpisów.

In [24]:
df.loc[df['settlement_status'] == "COMPLETE", 'loan_status'] = "Fully Paid"

In [25]:
df['loan_status'].value_counts()

Fully Paid     36246
Charged Off     6289
Name: loan_status, dtype: int64

###Usunięcie kolumn które zawierają informacje z przyszłości, niedostępne w momencie udzielania pożyczki

In [26]:
future_data = df[["mths_since_last_delinq", "last_pymnt_d",
                      "collection_recovery_fee", "recoveries", "issue_d",
                      "total_rec_late_fee", "total_rec_prncp", "funded_amnt",
                      "funded_amnt_inv", "last_fico_range_high",
                      "last_fico_range_low", "debt_settlement_flag"]]
selected_columns = future_data.columns
df.drop(columns=selected_columns, inplace=True, axis=1, errors='ignore')

###Usunięcie wierszy tam gdzie w kolumnach jest 1 NaN

In [78]:
selected_columns = nan_counts(df, 1, 1)
df = df.dropna(subset=selected_columns)

###Usunięcie kolumn ze zbędnymi informacjami

In [27]:
#Usunięcie kolumn które mają ponad 90% wartości NaN
selected_columns = nan_counts(df,0.9*df.shape[0], df.shape[0])
df.drop(selected_columns, inplace=True, axis=1, errors='ignore')

In [64]:
#Usunięcie kolumn z jedną unikalną wartością
selected_columns = unique_counts(df, 0, 2)
df.drop(selected_columns, inplace=True, axis=1, errors='ignore')

In [29]:
selected_columns = [value for value in unique_counts(df, 3, 100) if value in nan_counts(df, 1, df.shape[0])]
selected_columns

['emp_length',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'total_acc']

In [32]:
len(df.columns.tolist())

117

In [63]:
len(unique_counts(df, 0, 1))

0

In [65]:
[df[col].unique().tolist() for col in unique_counts(df, 0, 1)]

[]

In [79]:
len(nan_counts(df, 1, 1))

0

In [69]:
len(diff_counts(df, 1, df.shape[0]))

30

###Uzupełnienie modą kolumn float o małej ilości unikalnych wartości, o dużej róznicy między wartościami, małej ilości NaN

In [86]:
selected_columns = list(set(unique_counts(df, 0, 0.01*df.shape[0])) &
                        set(nan_counts(df, 1, 0.01*df.shape[0])) &
                        set(diff_counts(df, 0.1*df.shape[0], df.shape[0]))&
                        set([col for col in df.columns if df[col].dtype == 'float64']))

print(selected_columns)
print(len(selected_columns))
display_info(df, selected_columns, df_info)

df[selected_columns].fillna(df[selected_columns].mode().iloc[0], inplace=True)

['pub_rec', 'delinq_2yrs', 'inq_last_6mths']
3
Column Name: pub_rec
Description: Number of derogatory public records

Statistics:
count    42506.000000
mean         0.058156
std          0.245713
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          5.000000
Name: pub_rec, dtype: float64

Mode:
0    0.0
Name: pub_rec, dtype: float64

NaN count: 29

Unique values len:
7

Unique values:
[ 0.  1.  2.  3.  4.  5. nan]

Value counts:
0.0    40130
1.0     2298
2.0       64
3.0       11
4.0        2
5.0        1
Name: pub_rec, dtype: int64

Column Name: delinq_2yrs
Description: The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years

Statistics:
count    42506.000000
mean         0.152449
std          0.512406
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         13.000000
Name: delinq_2yrs, dtype: float64

Mode:
0    0.0
Name: delinq_2yrs, dtype: f

<ipython-input-86-91edf8fb999f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_columns].fillna(df[selected_columns].mode().iloc[0], inplace=True)
<ipython-input-86-91edf8fb999f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_columns].fillna(df[selected_columns].mode().iloc[0], inplace=True)
<ipython-input-86-91edf8fb999f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [80]:
selected_columns = list(set(unique_counts(df, 30, df.shape[0])) &
                        set(nan_counts(df, 1, 0.01*df.shape[0])) &
                        set(diff_counts(df, 1, df.shape[0]))&
                        set([col for col in df.columns if df[col].dtype == 'float64']))

print(selected_columns)
print(len(selected_columns))
display_info(df, selected_columns, df_info)

['total_acc', 'open_acc', 'annual_inc']
3
Column Name: total_acc
Description: The total number of credit lines currently in the borrower's credit file

Statistics:
count    42506.000000
mean        22.124406
std         11.592811
min          1.000000
25%         13.000000
50%         20.000000
75%         29.000000
max         90.000000
Name: total_acc, dtype: float64

Mode:
0    15.0
Name: total_acc, dtype: float64

NaN count: 29

Unique values len:
84

Unique values:
[ 9.  4. 10. 37. 38. 12. 11. 13.  3. 23. 34. 29. 28. 42. 14. 22. 21. 17.
  7. 31. 44. 26. 16.  6. 18. 27. 24. 25. 40. 35.  8. 20. 15. 19. 36. 51.
 32. 30. 33. 46.  5. 61. 56. 50. 41. 39. 79. 62. 43. 47. 53. 45. 60. 55.
 52. 58. 54. 57. 49. 63. 48. 59. 77. 87. 75. 72. 64. 67. 78. 76. 74. 66.
 81. 90. 80. 71. 69. 73. 70. 68. 65.  2.  1. nan]

Value counts:
15.0    1552
16.0    1547
17.0    1543
14.0    1531
20.0    1504
        ... 
74.0       1
77.0       1
78.0       1
72.0       1
90.0       1
Name: total_acc, Length: 

In [31]:
df_type_column['raw_column_type'].value_counts()

NameError: ignored

In [ ]:
for col_name in df.columns.tolist():
  if df[col_name].dtype == 'object':
    value_counts = df[col_name].value_counts()
    if len(value_counts) > 0.2*df.shape[0] and value_counts.iloc[1] < 10:
      display(value_counts)
      print("\n" + "=" * 40)

In [ ]:
selected_columns = {'url','id'}
df.drop(selected_columns, inplace=True, axis=1, errors='ignore')

###Przekształcenie wartości w kolumnach - usunięcie procentów, dodatkowych znaków, itd

In [ ]:
#Numeric
df["revol_util"] = df["revol_util"].str[0:-1].astype(np.float64)
df["int_rate"] = df["int_rate"].str[0:-1].astype(np.float64)

#Date
df['last_credit_pull_d'] = pd.to_datetime(df['last_credit_pull_d'], errors='coerce')
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], errors='coerce')

df['last_credit_pull_d_year'] = df['last_credit_pull_d'].dt.year
df['earliest_cr_line_year'] = df['earliest_cr_line'].dt.year

#Object
df["zip_code"] = df["zip_code"].str[0:-2]

In [ ]:
print("df after cleaning:", df.shape)

In [ ]:
selected_columns = df.columns.tolist()
for column_name in selected_columns:
    if column_name in df_info.index:
        description = df_info.loc[column_name, 'Description']
        print(f"{column_name} - {description}")
    else:
        print(f"Kolumna '{column_name}' nie istnieje w df_info.")

###Analiza brakujących wartości wraz z ich uzupełnieniem/usunięciem przy wzięciu pod uwagę pewnych ustalonych kryteriów

####"pub_rec"

In [ ]:
#Wypełnianie zerami wpisów, w których zmienna pubrec nie zawera informacji o
#negatywnych wpisach w rejestrze publicznym
df["pub_rec"].fillna(0, inplace=True)

####"pub_rec_bankruptcies"

In [ ]:
#Wypełnienie zerami kolumny nt. bankructw wpisanych do rejestru publicznego
df["pub_rec_bankruptcies"].fillna(0, inplace=True)

#### "emp_length"

In [ ]:
#Uzupełnianie pustych wartości modą
df['emp_length'].fillna(df['emp_length'].mode().iloc[0], inplace=True)

#### "emp_title"

In [ ]:
#Z racji dużej ilości pustych wpisów tej cechy, oraz fakt, że jest to zmienna
#typu 'object' oraz ma dużo unikalnych wartości to postanawiam tą zmienną usunąć
display_info(df, ['emp_title'], df_info)
df.drop('emp_title', inplace=True, axis=1)

#### "open_acc"

In [ ]:
display_info(df, ['open_acc'], df_info)
#Uzupełnianie pustych wartości modą
df["open_acc"].fillna(df["open_acc"].mode().iloc[0], inplace=True)

#### "purpose"

In [ ]:
display_info(df, ['purpose'], df_info)
df["purpose"].fillna(df["purpose"].mode().iloc[0], inplace=True)

#### "revol_util"

In [ ]:
display_info(df, ['revol_util'], df_info)
df["revol_util"].fillna(df["revol_util"].mode().iloc[0], inplace=True)

#### "last_credit_pull_d"

In [ ]:
display_info(df, ['last_credit_pull_d'], df_info)
df["last_credit_pull_d"].fillna(df["last_credit_pull_d"].mode().iloc[0], inplace=True)

#### "annual_inc"

In [ ]:
display_info(df, ['annual_inc'], df_info)
df["annual_inc"].fillna(df["annual_inc"].mode().iloc[0], inplace=True)

#### "loan_amnt"

In [ ]:
display_info(df, ['loan_amnt'], df_info)
df["loan_amnt"].fillna(df["loan_amnt"].mode().iloc[0], inplace=True)

#### "loan_status"

In [ ]:
display_info(df, ['loan_status'], df_info)
df["loan_status"].fillna(df["loan_status"].mode().iloc[0], inplace=True)

#### "fico_range_low"

In [ ]:
display_info(df, ['fico_range_low'], df_info)
df["fico_range_low"].fillna(df["fico_range_low"].mode().iloc[0], inplace=True)

#### "fico_range_high"

In [ ]:
display_info(df, ['fico_range_high'], df_info)
df["fico_range_high"].fillna(df["fico_range_high"].mode().iloc[0], inplace=True)

####fico_range_low oraz fico_range_high

In [ ]:
df[['fico_range_low','fico_range_high']].corr()

In [ ]:
#Te wartości sa mocno skorelowane więc zamiast dwóch kolumn tworzę jedną z ich średnich
df['fico_mean'] = (df.fico_range_low + df.fico_range_high) / 2
df.drop(['fico_range_low', 'fico_range_high'], axis=1, inplace=True)

###Uzupełnienie modą pozostałych kolumn o NaN < 10

In [ ]:
mask = df.isna()
naan_counts_columns = [(mask[column] == True).sum() for column in df.columns]
naan_column = pd.DataFrame([naan_counts_columns], columns=df.columns, index=['NaN'])
display(naan_column)

In [ ]:
selected_columns = [col_name for col_name in naan_column.columns if ((naan_column[col_name] > 0) & (naan_column[col_name] < 10)).all()]
for col_name in selected_columns:
    if df[col_name].dtype == 'object':
        df[col_name].fillna(df[col_name].mode().iloc[0], inplace=True)
    elif df[col_name].dtype == np.float64:
        df[col_name].fillna(df[col_name].mean(), inplace=True)


In [ ]:
selected_columns = [col_name for col_name in naan_column.columns if ((naan_column[col_name] > 10) & (naan_column[col_name] < 0.1*df.shape[0])).all()]

####"title"

In [ ]:
display_info(df, ["title"], df_info)
df.drop("title", inplace=True, axis=1, errors='ignore')

####"delinq_2yrs"

####"earliest_cr_line"

In [ ]:
display_info(df, ["earliest_cr_line"], df_info)
df["earliest_cr_line"].fillna(df["earliest_cr_line"].mode().iloc[0], inplace=True)

In [ ]:
display_info(df, ["delinq_2yrs"], df_info)
df["delinq_2yrs"].fillna(df["delinq_2yrs"].mode().iloc[0], inplace=True)

####"inq_last_6mths"

In [ ]:
display_info(df, ["inq_last_6mths"], df_info)
df["inq_last_6mths"].fillna(df["inq_last_6mths"].mode().iloc[0], inplace=True)

####"total_acc"

In [ ]:
display_info(df, ["total_acc"], df_info)
df["total_acc"].fillna(df["total_acc"].mode().iloc[0], inplace=True)

####"acc_now_delinq"

In [ ]:
display_info(df, ["acc_now_delinq"], df_info)
df.drop("acc_now_delinq", inplace=True, axis=1, errors='ignore')

####"delinq_amnt"

In [ ]:
display_info(df, ["delinq_amnt"], df_info)
df.drop("delinq_amnt", inplace=True, axis=1, errors='ignore')

####"tax_liens"

In [ ]:
display_info(df, ["tax_liens"], df_info)
df.drop("tax_liens", inplace=True, axis=1, errors='ignore')

#### Usunięcie pozostałych kolumn o dużej ilości NaN

In [ ]:
selected_columns = [col_name for col_name in naan_column.columns if (naan_column[col_name] >  0.1*df.shape[0]).all()]
display_info(df, selected_columns, df_info)
df.drop(selected_columns, inplace=True, axis=1, errors='ignore')

###Podsumowanie


In [ ]:
mask = df.isna()
naan_counts_columns = [(mask[column] == True).sum() for column in df.columns]
naan_column = pd.DataFrame([naan_counts_columns], columns=df.columns, index=['NaN'])
display(naan_column)